In [1]:
import pandas as pd
df_all=pd.read_csv('C:/diabetic_data_processed_withweight.csv',';')
df_all=df_all.groupby(['patient_nbr'], sort=True).last()
print(type(df_all))
to_del = ['admission_source_id','encounter_id', 'patient_nbr','medical_specialty','payer_code','index','admission_type_id','discharge_disposition_id',
          'nateglinide','chlorpropamide','acetohexamide','tolbutamide','acarbose','miglitol','troglitazone','tolazamide','examide','citoglipton','glyburide-metformin','glipizide-metformin','glimepiride-pioglitazone','metformin-rosiglitazone','metformin-pioglitazone']        
print (to_del)
#Filter_selected cols
filtered_cols = [c for c in df_all.columns if (c not in to_del) ]#and ('ENF' not in c)
df_2 = df_all[filtered_cols]
print ("df_2",df_2.shape)

<class 'pandas.core.frame.DataFrame'>
['admission_source_id', 'encounter_id', 'patient_nbr', 'medical_specialty', 'payer_code', 'index', 'admission_type_id', 'discharge_disposition_id', 'nateglinide', 'chlorpropamide', 'acetohexamide', 'tolbutamide', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton', 'glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone']
df_2 (71518, 28)


In [2]:
from sklearn.model_selection import train_test_split
import numpy as np
df_3 = df_2.fillna(value=np.mean(df_2,axis=0),inplace=False,axis=0).values
print ("df_3",df_3.shape)
print(df_2.columns)
X=df_3[:,0:-1]
y=df_3[:,-1]
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train=np.where(y_train > 0, 1, y_train)
y_test=np.where(y_test > 0, 1, y_test)
print(set(y_train))
print(set(y_test))
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)
from imblearn.over_sampling import SMOTE
sm = SMOTE(n_jobs=-1, random_state=42,kind='regular')
x_train_res,y_train_res = sm.fit_sample(x_train,y_train)

df_3 (71518, 28)
Index(['race', 'gender', 'age', 'weight', 'time_in_hospital',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'glimepiride', 'glipizide', 'glyburide',
       'pioglitazone', 'rosiglitazone', 'insulin', 'change', 'diabetesMed',
       'readmitted'],
      dtype='object')
{0.0, 1.0}
{0.0, 1.0}
(57214, 27)
(14304, 27)
(57214,)
(14304,)


In [3]:
from sklearn.preprocessing import StandardScaler
# Define the scaler 
scaler = StandardScaler().fit(x_train_res)
# Scale the train set
X_train = scaler.transform(x_train_res)
# Scale the test set
X_test = scaler.transform(x_test)
y_train=y_train_res

In [4]:
# Import `Sequential` from `keras.models`
from keras.models import Sequential
# Import `Dense` from `keras.layers`
from keras.layers import Dense
# Initialize the constructor
model = Sequential()
# Add an input layer 
model.add(Dense(500, activation='relu', input_shape=(27,)))
# Add one hidden layer 
model.add(Dense(500, activation='relu'))
# Add an output layer 
model.add(Dense(1, activation='sigmoid'))
# Model output shape
model.output_shape
# Model summary
model.summary()
# Model config
model.get_config()
# List all weight tensors 
model.get_weights()

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 500)               14000     
_________________________________________________________________
dense_2 (Dense)              (None, 500)               250500    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 501       
Total params: 265,001
Trainable params: 265,001
Non-trainable params: 0
_________________________________________________________________


[array([[-0.02457942, -0.0738682 ,  0.03116889, ..., -0.05410603,
         -0.08052338,  0.01208315],
        [-0.07220162,  0.06896602,  0.03888202, ..., -0.00595117,
         -0.03931887,  0.07936023],
        [-0.07824547,  0.08367039,  0.00159086, ..., -0.08465236,
         -0.07461681, -0.06413803],
        ..., 
        [-0.01367509,  0.0346559 , -0.10064573, ..., -0.0221258 ,
          0.06622039,  0.00564953],
        [-0.10457325, -0.08996287, -0.09190982, ...,  0.08690262,
          0.06886707,  0.0450476 ],
        [-0.05272318,  0.07545906, -0.09247122, ...,  0.04745248,
         -0.04506616, -0.08348849]], dtype=float32),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [17]:
import tensorflow as tf
def auc_roc(y_true, y_pred):
    # any tensorflow metric
    value, update_op = tf.contrib.metrics.streaming_auc(y_pred, y_true)

    # find all variables created for this metric
    metric_vars = [i for i in tf.local_variables() if 'auc_roc' in i.name.split('/')[1]]

    # Add metric variables to GLOBAL_VARIABLES collection.
    # They will be initialized for new session.
    for v in metric_vars:
        tf.add_to_collection(tf.GraphKeys.GLOBAL_VARIABLES, v)

    # force to update metric values
    with tf.control_dependencies([update_op]):
        value = tf.identity(value)
        return value

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[auc_roc])            
model.fit(X_train, y_train,epochs=20, batch_size=1, verbose=1)

Epoch 1/20
86872/86872 [==============================] - 622s - loss: 0.4786 - auc_roc: 0.8307   
Epoch 2/20
86872/86872 [==============================] - 602s - loss: 0.4667 - auc_roc: 0.8354   
Epoch 3/20
86872/86872 [==============================] - 608s - loss: 0.4598 - auc_roc: 0.8384   
Epoch 4/20
86872/86872 [==============================] - 594s - loss: 0.4527 - auc_roc: 0.8410   
Epoch 5/20
86872/86872 [==============================] - 587s - loss: 0.4489 - auc_roc: 0.8433   
Epoch 6/20
86872/86872 [==============================] - 578s - loss: 0.4440 - auc_roc: 0.8452   
Epoch 7/20
86872/86872 [==============================] - 577s - loss: 0.4403 - auc_roc: 0.8471   
Epoch 8/20
86872/86872 [==============================] - 379s - loss: 0.4369 - auc_roc: 0.8487   
Epoch 9/20
86872/86872 [==============================] - 374s - loss: 0.4326 - auc_roc: 0.8505   
Epoch 10/20
86872/86872 [==============================] - 358s - loss: 0.4302 - auc_roc: 0.8521   
Epoch 11/

In [18]:
# Import the modules from `sklearn.metrics`
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, cohen_kappa_score
y_pred = model.predict(X_test)
score = model.evaluate(X_test, y_test,verbose=1)
print(score)

# Confusion matrix
print(confusion_matrix(y_test, y_pred.round()))
       
# Precision 
print("Precision: ",precision_score(y_test, y_pred.round()))

# Recall
print("Recall: ",recall_score(y_test, y_pred.round()))

# F1 score
print("F1score: ", f1_score(y_test,y_pred.round()))

# Cohen's kappa
print("Cohen's kappa",cohen_kappa_score(y_test, y_pred.round()))


13696/14304 [===========================>..] - ETA: 0s[0.61511648234638328, 0.8648078701640135]
[[9622 1293]
 [2497  892]]
Precision:  0.40823798627
Recall:  0.263204485099
F1score:  0.320057409401
Cohen's kappa 0.1649462822


In [19]:
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred.round())
roc_auc = auc(fpr, tpr)
print(roc_auc)

0.572371825692


In [20]:
matrix=confusion_matrix(y_test, y_pred.round())
specificity=matrix[1,1]/(matrix[1,1]+matrix[0,1])
print(specificity)

0.40823798627
